In [1]:
import pandas as pd
import numpy as np

import genetic_algorithm as ga
from nn_ga import GenAlgNN

In [2]:
spiral = pd.read_csv('../two_spirals.csv', names=['x', 'y', 'class'])

In [3]:
X_valid = spiral.copy()
y_valid = X_valid.pop('class')

X_train = spiral.sample(frac=0.5)
y_train = X_train.pop('class')

In [4]:
X_valid, y_valid

(           x        y
 0    4.47485 -0.89004
 1   -4.47485  0.89004
 2    4.50000  0.00007
 3   -4.50000 -0.00007
 4    4.35222  0.86578
 ..       ...      ...
 259 -0.57744  0.33915
 260  0.55170 -0.00971
 261 -0.55170  0.20971
 262  0.50000  0.10002
 263 -0.50000  0.09998
 
 [264 rows x 2 columns],
 0      1
 1      0
 2      1
 3      0
 4      1
       ..
 259    0
 260    1
 261    0
 262    1
 263    0
 Name: class, Length: 264, dtype: int64)

In [5]:
X_train, y_train

(           x        y
 222 -1.23748  1.33739
 207 -1.59094 -1.49104
 215  0.00007 -1.90000
 117 -0.18293  0.91948
 114  0.00004 -1.00000
 ..       ...      ...
 89   1.00702 -1.50700
 73  -1.92273 -1.28482
 11  -3.00515 -3.00525
 17  -0.79248 -3.98445
 137 -4.35222 -0.76578
 
 [132 rows x 2 columns],
 222    1
 207    0
 215    0
 117    0
 114    1
       ..
 89     0
 73     0
 11     0
 17     0
 137    0
 Name: class, Length: 132, dtype: int64)

In [6]:
problem_ga = GenAlgNN((np.array(X_train), np.array(y_train)), (np.array(X_valid), np.array(y_valid)))
ga_optimizer = ga.GAOptimizer(problem_ga)

In [7]:
ga_optimizer.start(10)

In [8]:
for _ in range(10):
    gen = ga_optimizer.next_generation()
    print('=== New generation ===')

=== New generation ===
=== New generation ===
=== New generation ===
=== New generation ===
=== New generation ===
=== New generation ===
=== New generation ===
=== New generation ===
=== New generation ===
=== New generation ===


In [11]:
fitnesses = [(problem_ga.fitness(i), i) for i in gen]
mse = [(1.0 / f - 1.0, i) for f, i in fitnesses]
mse.sort(key=lambda t: t[0])

In [12]:
mse

[(0.252246230840683, [3, 12, 11, 3, 14, 14, 0, 8]),
 (0.2536109685897827, [3, 3, 11, 3, 2, 2, 0, 8]),
 (0.2552160918712616, [2, 8, 11, 3, 9, 14, 0, 8]),
 (0.25781360268592834, [2, 3, 11, 3, 5, 14, 0, 8]),
 (0.26478826999664307, [3, 12, 6, 3, 9, 14, 12, 15]),
 (0.27595144510269165, [3, 12, 11, 3, 14, 14, 0, 8]),
 (0.278186172246933, [3, 12, 11, 3, 14, 14, 12, 15]),
 (0.28510403633117676, [3, 12, 10, 3, 9, 14, 12, 15]),
 (0.2931174039840698, [3, 12, 11, 3, 14, 14, 0, 8]),
 (0.312226265668869, [15, 12, 12, 3, 14, 14, 0, 8])]